In [ ]:
# Import agents
from trust_bayesian_agent_comparison.agents import FocalAgent, BayesianFocalAgent

# Import partners
from trust_bayesian_agent_comparison.partners import (
    TitForTatCooperatePartner,
    AlwaysDefectPartner,
    StrategicCheaterPartner,
)

# Import simulation
from trust_bayesian_agent_comparison.simulation import run_agent_simulation

# Import analysis tools
from trust_bayesian_agent_comparison.analysis import (
    SensitivityAnalysisManager,
    MonteCarloManager,
    compute_strategy_statistics,
)

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

## Example 1: Single Simulation Run

In [ ]:
# Create agents
trust_agent = FocalAgent(loss_aversion=2.0, mu=0.5)
bayesian_agent = BayesianFocalAgent(alpha_0=0.5, beta_0=0.5)

# Create partner
partner = TitForTatCooperatePartner()

# Run simulation
df_trust = run_agent_simulation(trust_agent, partner, num_rounds=70, seed=42)

# Compute statistics
stats = compute_strategy_statistics(df_trust)
print("Trust Agent vs TitForTat:")
for key, value in stats.items():
    print(f"  {key}: {value:.3f}")

In [ ]:
# Plot belief evolution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(df_trust['agent_belief'], label='Trust Agent', color='#2E86AB')
plt.axhline(y=2/3, color='red', linestyle='--', label='Threshold', alpha=0.5)
plt.xlabel('Round')
plt.ylabel('Belief')
plt.title('Belief Evolution')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(df_trust['agent_action'], label='Agent', marker='o', linestyle='')
plt.plot(df_trust['partner_action'], label='Partner', marker='x', linestyle='')
plt.xlabel('Round')
plt.ylabel('Action (1=Coop, 0=Defect)')
plt.title('Actions Over Time')
plt.legend()

plt.tight_layout()
plt.show()

## Example 2: Sensitivity Analysis

Run parameter sweeps with automatic result management.

In [ ]:
# Initialize manager
sensitivity_manager = SensitivityAnalysisManager()

# Run analysis for TitForTat (auto-saves with timestamp)
results_tft = sensitivity_manager.run_analysis(
    partner_name="TitForTatCoop",
    partner_factory=lambda: TitForTatCooperatePartner(),
    threshold_direction="up",
    overwrite=False,  # Load existing if available
)

print(f"Loaded {len(results_tft)} simulation results")
print(f"Columns: {list(results_tft.columns)}")

In [ ]:
# Run for multiple partners
partner_configs = [
    ("TitForTatCoop", lambda: TitForTatCooperatePartner(), "up"),
    ("AlwaysDefect", lambda: AlwaysDefectPartner(), "down"),
    ("StrategicCheat", lambda: StrategicCheaterPartner(coop_length=5), "down"),
]

all_results = sensitivity_manager.run_multiple(
    partner_configs=partner_configs,
    overwrite=False,
)

for partner_name, df in all_results.items():
    print(f"{partner_name}: {len(df)} rows")

## Example 3: Monte Carlo Comparison

Compare trust-based vs Bayesian agents over many runs.

In [ ]:
# Initialize manager
mc_manager = MonteCarloManager()

# Run Monte Carlo simulation
df_trust, df_bayesian = mc_manager.run_monte_carlo(
    agent1_factory=lambda: FocalAgent(loss_aversion=2.0, mu=0.5),
    agent2_factory=lambda: BayesianFocalAgent(alpha_0=0.5, beta_0=0.5),
    partner_factory=lambda: TitForTatCooperatePartner(),
    partner_name="TitForTatCoop",
    num_runs=100,  # Fewer runs for quick demo
    overwrite=False,
)

print(f"Trust agent: {len(df_trust)} total rounds")
print(f"Bayesian agent: {len(df_bayesian)} total rounds")

In [ ]:
# Compare payoffs
trust_payoffs = df_trust.groupby('run_id')['agent_payoff'].sum()
bayesian_payoffs = df_bayesian.groupby('run_id')['agent_payoff'].sum()

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.hist(trust_payoffs, bins=20, alpha=0.6, label='Trust', color='#2E86AB')
plt.hist(bayesian_payoffs, bins=20, alpha=0.6, label='Bayesian', color='#A23B72')
plt.xlabel('Total Payoff')
plt.ylabel('Frequency')
plt.title('Payoff Distribution')
plt.legend()

plt.subplot(1, 2, 2)
plt.boxplot([trust_payoffs, bayesian_payoffs], labels=['Trust', 'Bayesian'])
plt.ylabel('Total Payoff')
plt.title('Payoff Comparison')

plt.tight_layout()
plt.show()

print(f"Trust mean: {trust_payoffs.mean():.2f} ± {trust_payoffs.std():.2f}")
print(f"Bayesian mean: {bayesian_payoffs.mean():.2f} ± {bayesian_payoffs.std():.2f}")

## Benefits of Refactored Structure

1. **Clean imports**: No need to copy-paste cell definitions
2. **Automatic organization**: Results saved in dated folders
3. **Overwrite control**: Explicit `overwrite=False` loads existing results
4. **Reproducibility**: Timestamped results for version tracking
5. **Scalability**: Easy to add new partners/agents
6. **CLI support**: Can run experiments from command line

```bash
# Run from terminal
python scripts/run_sensitivity.py --partners TitForTatCoop AlwaysDefect
```